In [143]:
cursor.execute('d')

<module 'corrlab' from 'corrlab.py'>

In [60]:
cursor.execute("DROP DATABASE IF EXISTS db_corr")

0L

In [61]:
#-*- coding:utf-8 -*-
#need to input script like 'python corr_ana.py '20170910' '20170920' '5s'



# typelst = ['noble', 'nonferrous', 'black', 'farm', 'chemical', 'futures', 'loan']

import pandas as pd
import numpy as np
import re
import matplotlib.pylab as plt
import seaborn as sns
import os
import gc
import corrlab
import MySQLdb


def createTable():
    conn = MySQLdb.connect(host = 'localhost',user='root',passwd='hhui123456')
    cursor = conn.cursor()
    cursor.execute("""create database if not exists db_corr""")
    conn.select_db('db_corr')
    cursor.execute("""create table if not exists tb_corr(
    start_date DATE not null,
    end_date DATE not null,
    ticker1 varchar(32) not null,
    ticker2 varchar(32) not null,
    type SMALLINT not null DEFAULT 0, 
    period INT not null,
    lag INT not null,
    corr DOUBLE,
    symbol1 varchar(32),
    symbol2 varchar(32),
    primary key(start_date, end_date, ticker1, ticker2, type, lag, period)
    )""")

def calc_target(start_date, end_date, type, ticker1,lagLst = lagLst, periodLst = periodLst, filedir = filedir, database = 'db_corr'):
    corr = corrlab.corrAna(filedir=filedir, start_date=start_date, end_date=end_date, type=type)
    conn = MySQLdb.connect(host='localhost', user='root', passwd='hhui123456')
    cursor = conn.cursor()
    conn.select_db(database)
    dayLst = corr.generateDayLst()
    for day in dayLst:   # 时间跨度为1天
        print 'processing day:', day
        data = corr.concatdata([day])
        symbol1 = corr.symbolDict[day][ticker1[:2]]
        for lag in lagLst:
            for period in periodLst:
                res = pd.DataFrame()
                temp = data.copy()
                shifted = temp[ticker1].shift(-int(lag[:-1]), 's')
                align_base = corr.get_align_base(data)
                _, align_shifted = align_base.align(shifted, join='left', axis=0)
                temp[ticker1] = align_shifted.values
                temp = corr.sampledata(temp, period = period)
                temp.fillna(method='ffill',inplace=True)
                temp.fillna(method = 'bfill',inplace=True)
                temp_corr = temp.corr().sort_index()
                res = pd.concat([res,temp_corr[ticker1]])
                res.rename(columns = {0:day},inplace=True)
                res.fillna(-2,inplace=True)
                print 'lag is : %s, period is %s, type is %d' %(lag, period, type)
                for ticker2 in res.index.values:
                    corr_value = res[day][ticker2]
                    ticker2 = ticker2.split('_')[0]
                    symbol2 = corr.symbolDict[day][ticker2[:2]]
                    cursor.execute("""REPLACE INTO tb_corr(
                                start_date,
                                end_date,
                                ticker1,
                                symbol1,
                                ticker2,
                                symbol2,
                                type,
                                period,
                                lag,
                                corr)
                                VALUES (
                                '%s', '%s','%s','%s','%s','%s','%d','%d','%d','%.6f'
                                )
                                """ % (day, day, ticker1, symbol1, ticker2, symbol2, type,int(period[:-1]), int(lag[:-1]), corr_value))
                    conn.commit()



/home/hui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:30: Warning: Can't create database 'db_corr'; database exists
/home/hui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:44: Warning: Table 'tb_corr' already exists


processing day: 20171101
lag is : 1s, period is 1s, type is 0
lag is : 1s, period is 5s, type is 0
lag is : 1s, period is 10s, type is 0
lag is : 1s, period is 30s, type is 0
lag is : 1s, period is 60s, type is 0
lag is : 5s, period is 1s, type is 0
lag is : 5s, period is 5s, type is 0
lag is : 5s, period is 10s, type is 0
lag is : 5s, period is 30s, type is 0
lag is : 5s, period is 60s, type is 0
lag is : 10s, period is 1s, type is 0
lag is : 10s, period is 5s, type is 0
lag is : 10s, period is 10s, type is 0
lag is : 10s, period is 30s, type is 0
lag is : 10s, period is 60s, type is 0
lag is : 30s, period is 1s, type is 0
lag is : 30s, period is 5s, type is 0
lag is : 30s, period is 10s, type is 0
lag is : 30s, period is 30s, type is 0
lag is : 30s, period is 60s, type is 0
lag is : 60s, period is 1s, type is 0
lag is : 60s, period is 5s, type is 0
lag is : 60s, period is 10s, type is 0
lag is : 60s, period is 30s, type is 0
lag is : 60s, period is 60s, type is 0
processing day: 2017

lag is : 10s, period is 5s, type is 0
lag is : 10s, period is 10s, type is 0
lag is : 10s, period is 30s, type is 0
lag is : 10s, period is 60s, type is 0
lag is : 30s, period is 1s, type is 0
lag is : 30s, period is 5s, type is 0
lag is : 30s, period is 10s, type is 0
lag is : 30s, period is 30s, type is 0
lag is : 30s, period is 60s, type is 0
lag is : 60s, period is 1s, type is 0
lag is : 60s, period is 5s, type is 0
lag is : 60s, period is 10s, type is 0
lag is : 60s, period is 30s, type is 0
lag is : 60s, period is 60s, type is 0
processing day: 20171114
lag is : 1s, period is 1s, type is 0
lag is : 1s, period is 5s, type is 0
lag is : 1s, period is 10s, type is 0
lag is : 1s, period is 30s, type is 0
lag is : 1s, period is 60s, type is 0
lag is : 5s, period is 1s, type is 0
lag is : 5s, period is 5s, type is 0
lag is : 5s, period is 10s, type is 0
lag is : 5s, period is 30s, type is 0
lag is : 5s, period is 60s, type is 0
lag is : 10s, period is 1s, type is 0
lag is : 10s, period

lag is : 60s, period is 10s, type is 0
lag is : 60s, period is 30s, type is 0
lag is : 60s, period is 60s, type is 0
processing day: 20171124
lag is : 1s, period is 1s, type is 0
lag is : 1s, period is 5s, type is 0
lag is : 1s, period is 10s, type is 0
lag is : 1s, period is 30s, type is 0
lag is : 1s, period is 60s, type is 0
lag is : 5s, period is 1s, type is 0
lag is : 5s, period is 5s, type is 0
lag is : 5s, period is 10s, type is 0
lag is : 5s, period is 30s, type is 0
lag is : 5s, period is 60s, type is 0
lag is : 10s, period is 1s, type is 0
lag is : 10s, period is 5s, type is 0
lag is : 10s, period is 10s, type is 0
lag is : 10s, period is 30s, type is 0
lag is : 10s, period is 60s, type is 0
lag is : 30s, period is 1s, type is 0
lag is : 30s, period is 5s, type is 0
lag is : 30s, period is 10s, type is 0
lag is : 30s, period is 30s, type is 0
lag is : 30s, period is 60s, type is 0
lag is : 60s, period is 1s, type is 0
lag is : 60s, period is 5s, type is 0
lag is : 60s, period

lag is : 5s, period is 10s, type is 0
lag is : 5s, period is 30s, type is 0
lag is : 5s, period is 60s, type is 0
lag is : 10s, period is 1s, type is 0
lag is : 10s, period is 5s, type is 0
lag is : 10s, period is 10s, type is 0
lag is : 10s, period is 30s, type is 0
lag is : 10s, period is 60s, type is 0
lag is : 30s, period is 1s, type is 0
lag is : 30s, period is 5s, type is 0
lag is : 30s, period is 10s, type is 0
lag is : 30s, period is 30s, type is 0
lag is : 30s, period is 60s, type is 0
lag is : 60s, period is 1s, type is 0
lag is : 60s, period is 5s, type is 0
lag is : 60s, period is 10s, type is 0
lag is : 60s, period is 30s, type is 0
lag is : 60s, period is 60s, type is 0
processing day: 20171207
lag is : 1s, period is 1s, type is 0
lag is : 1s, period is 5s, type is 0
lag is : 1s, period is 10s, type is 0
lag is : 1s, period is 30s, type is 0
lag is : 1s, period is 60s, type is 0
lag is : 5s, period is 1s, type is 0
lag is : 5s, period is 5s, type is 0
lag is : 5s, period 

In [2]:
import MySQLdb
conn = MySQLdb.connect(host='localhost', user='root', passwd='hhui123456')
cursor = conn.cursor()
conn.select_db('db_corr')
cursor.execute('select * from tb_corr where type = 0')

64650L

In [8]:
cursor.execute('select * from tb_corr where type = 0 and start_date = "20171114" and period = 1 and lag = 1 and ticker1 = "ru0"')

39L

In [9]:
rec = cursor.fetchall()
for i in rec:
    print i

(datetime.date(2017, 11, 14), datetime.date(2017, 11, 14), 'ru0', 'a10', 0, 1L, 1L, 0.337286, 'ru1801', 'a1801')
(datetime.date(2017, 11, 14), datetime.date(2017, 11, 14), 'ru0', 'ag0', 0, 1L, 1L, 0.005093, 'ru1801', 'ag1712')
(datetime.date(2017, 11, 14), datetime.date(2017, 11, 14), 'ru0', 'al0', 0, 1L, 1L, 0.434727, 'ru1801', 'al1801')
(datetime.date(2017, 11, 14), datetime.date(2017, 11, 14), 'ru0', 'au0', 0, 1L, 1L, 0.009789, 'ru1801', 'au1712')
(datetime.date(2017, 11, 14), datetime.date(2017, 11, 14), 'ru0', 'bu0', 0, 1L, 1L, 0.018287, 'ru1801', 'bu1712')
(datetime.date(2017, 11, 14), datetime.date(2017, 11, 14), 'ru0', 'c10', 0, 1L, 1L, -0.384815, 'ru1801', 'c1801')
(datetime.date(2017, 11, 14), datetime.date(2017, 11, 14), 'ru0', 'CF0', 0, 1L, 1L, 0.004938, 'ru1801', 'CF801')
(datetime.date(2017, 11, 14), datetime.date(2017, 11, 14), 'ru0', 'cs0', 0, 1L, 1L, 0.010371, 'ru1801', 'cs1801')
(datetime.date(2017, 11, 14), datetime.date(2017, 11, 14), 'ru0', 'cu0', 0, 1L, 1L, 0.0443